# Agent Evaluation with RAGAS

We will use the RAGAS library to evaluate the performance our Gemini agent. RAGAS is a framework designed to assess the capabilities of AI agents in various tasks.

This notebook uses the functions in `ragas_eval.py` to evaluate our RAG pipeline
across different hyperparameter combinations.

The parameters for the evaluation are as follows:
* chunk_size
* top_k
* temperature

In [6]:
%load_ext autoreload
%autoreload 2

## Montar en Colab

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [36]:
ROOT_DIR = "/content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/model_evaluation"
%cd $ROOT_DIR

/content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/model_evaluation


In [3]:
%ls $ROOT_DIR

Evaluate_Model.ipynb  ragas_eval.py


In [4]:
!pip install langchain-community langchain-google-genai duckduckgo-search pypdf faiss-cpu python-dotenv ragas pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 

In [3]:
# Imports
import sys
sys.path.append('..')

import os
import sys
import warnings
import pandas as pd
from datetime import datetime

# Suprimir warnings innecesarios
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from generative_resp import pdf_process_utils
import ragas_eval as ragas_eval

project_root = os.path.dirname(ROOT_DIR)
sys.path.append(project_root)

In [10]:
def main():
    """
    Función principal que carga los documentos y ejecuta la evaluación.
    """
    print("\n" + "="*60)
    print("🚀 INICIANDO EVALUACIÓN RAG SIMPLIFICADA")
    print(f"📅 Fecha y Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("="*60)

    # Definir la ruta al directorio de los PDFs
    pdf_directory = os.path.join(project_root, 'generative_resp', 'polizas')

    if not os.path.isdir(pdf_directory):
        print(f"❌ Error: El directorio de PDFs no se encuentra en: {pdf_directory}")
        return

    print(f"📁 Cargando documentos desde: {pdf_directory}...")

    # Cargar los documentos. La división inicial no es crítica, ya que cada
    # configuración la re-dividirá según su 'chunk_size'.
    all_documents = pdf_process_utils.load_split_pdfs(
        pdf_dir=pdf_directory,
        chunk_size=1000,
        chunk_overlap=100
    )

    if not all_documents:
        print("❌ No se pudieron cargar los documentos. Abortando.")
        return

    print(f"✅ {len(all_documents)} chunks de documentos cargados inicialmente.")

    # --- Ejecutar la evaluación ---
    print("\n▶️ Lanzando el proceso de evaluación de RAGAS...")

    try:
        results_df = ragas_eval.run_evaluation(original_docs=all_documents)

        if results_df is not None:
            print("\n" + "="*60)
            print("🎉 ¡EVALUACIÓN COMPLETADA!")
            print("="*60)
            print("📊 Resumen de Resultados Finales (Promedio por combinación):")

            # Imprimir el DataFrame de resultados con un formato más limpio
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 100)
            pd.set_option('display.colheader_justify', 'center')
            pd.set_option('display.precision', 4)

            print(results_df)

            # Guardar resultados en un archivo CSV
            results_path = 'ragas_simplified_results.csv'
            results_df.to_csv(results_path, index=False)
            print(f"\n💾 Resultados guardados en: {results_path}")

        else:
            print("\n❌ La evaluación no produjo resultados.")

    except Exception as e:
        print(f"\n❌ Ocurrió un error crítico durante la evaluación: {e}")
        import traceback
        traceback.print_exc()

In [37]:
if __name__ == "__main__":
    main()


🚀 INICIANDO EVALUACIÓN RAG SIMPLIFICADA
📅 Fecha y Hora: 2025-06-14 02:09:08
📁 Cargando documentos desde: /content/drive/MyDrive/Proyectos/Insurance-Chatbot/Insurance_Chatbot_project/api_ai/generative_resp/polizas...
Split 267 documents into 699 chunks.
✅ 699 chunks de documentos cargados inicialmente.

▶️ Lanzando el proceso de evaluación de RAGAS...
Configurando dependencias de RAGAS (LLM y Embeddings)...
✅ Dependencias de RAGAS configuradas exitosamente.

🧪 Evaluando Configuración #1/4: Baseline
   Hiperparámetros: chunk_size=1000, top_k=4, temp=0.1
  - Evaluando pregunta 1/4...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

    ✅ Éxito (resultado procesado desde lista).
    ⏳ Pausa de 180 segundos para proteger la cuota...
  - Evaluando pregunta 2/4...


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

    ✅ Éxito (resultado procesado desde lista).
    ⏳ Pausa de 180 segundos para proteger la cuota...
  - Evaluando pregunta 3/4...


...

    ⚠️ Fallo por excepción: 429 You exceeded your current quota, please check your plan and billing details. For more informatio...
    ⏳ Pausa de 180 segundos para proteger la cuota...


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 500
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
].


  - Evaluando pregunta 3/4...
    ⚠️ Fallo por excepción: 429 You exceeded your current quota, please check your plan and billing details. For more informatio...
    ⏳ Pausa de 180 segundos para proteger la cuota...


  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 500
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
].


  - Evaluando pregunta 4/4...
    ⚠️ Fallo por excepción: 429 You exceeded your current quota, please check your plan and billing details. For more informatio...
    ⏳ Pausa de 180 segundos para proteger la cuota...

  📊 Resultados promedio para la configuración:
     - faithfulness: Falló
     - answer_relevancy: Falló
     - context_precision: Falló
     - context_recall: Falló

🎉 ¡EVALUACIÓN COMPLETADA!
📊 Resumen de Resultados Finales (Promedio por combinación):
         combination_name        faithfulness  answer_relevancy  context_precision  context_recall
0                      Baseline     0.9896          0.6248             0.375            0.5000    
1         More Context, Focused     1.0000          0.6497             0.375            0.5000    
2  Smaller Chunks, More Options     0.8771          0.3177             0.125            0.3333    
3           Creative & Balanced        NaN             NaN               NaN               NaN    

💾 Resultados guardados en: ragas_s